In [2]:
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import re
import Preprocessing

labelencoder = LabelEncoder()
df = pd.read_csv('Poltifact_dataset.csv',sep=',',index_col='idx')
df = Preprocessing.Preprocess_text(df)
#df['Image'] = labelencoder.inverse_transform(df['Image'])
df['Labels'] = labelencoder.fit_transform(df['Labels'])
print(labelencoder.classes_)
df.head()

['FALSE' 'Half-True' 'Mostly False' 'Mostly True' 'Pants on Fire!' 'TRUE']


,Title,Date,Author,Article,Labels
idx,,,,,
0,no bill cosby wasnt framed for sexual assault ...,"on Monday, November 4th, 2019 at 2:31 p.m.",samantha putterman,actor and comedian bill cosby was convicted on...,4
1,green new deal doesnt mention abortion but nik...,"on Friday, November 1st, 2019 at 1:39 p.m.",tom kertscher,attacks on the green new deal include claims t...,2
2,subpoenas vs bills what this cnn pundit got wrong,"on Monday, November 4th, 2019 at 2:03 p.m.",bill mccarthy,cnn commentator mike shields took aim at house...,0
3,kamala harris didnt say democrats are coming f...,"on Monday, November 4th, 2019 at 1:03 p.m.",daniel funke,a quote from sen kamala harris attacking suppo...,4
4,nancy pelosi overstates republican gains in im...,"on Monday, November 4th, 2019 at 11:44 a.m.",jon greenberg,house republicans voted unanimously against a ...,2


In [32]:
train_all = df[['Author','Title','Article']]
target_all = df['Labels']
column = df['Title']
print(column[3])
train_all.head()

Kamala Harris didnt say Democrats are coming for Trump supporters in NUM


,Title,Article
idx,,
0,No Bill Cosby wasnt framed for sexual assault ...,Actor and comedian Bill Cosby was convicted on...
1,Green New Deal doesnt mention abortion but Nik...,Attacks on the Green New Deal include claims t...
2,Subpoenas vs bills What this CNN pundit got wrong,CNN commentator Mike Shields took aim at House...
3,Kamala Harris didnt say Democrats are coming f...,A quote from Sen Kamala Harris attacking suppo...
4,Nancy Pelosi overstates Republican gains in im...,House Republicans voted unanimously against a ...


In [33]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

max_length = max([len(s.split()) for s in train_all['Article']])
max_length2 = max([len(s.split()) for s in train_all['Title']])
max_length3 = max([len(s.split()) for s in train_all['Author']])
x_train, x_test, y_train, y_test = train_test_split(train_all,target_all, test_size=0.2, random_state=42)
tokenizer = Tokenizer()
tokenizer2 = Tokenizer()
tokenizer3 = Tokenizer()
tokenizer.fit_on_texts(list(train_all['Article']))
x_train_Article_tokens = tokenizer.texts_to_sequences(list(x_train['Article']))
x_test_Article_tokens = tokenizer.texts_to_sequences(list(x_test['Article']))
x_train_Article_pad = pad_sequences(x_train_Article_tokens,maxlen=max_length,padding='post')
x_test_Article_pad = pad_sequences(x_test_Article_tokens,maxlen=max_length,padding='post')

unique_words = len(tokenizer.word_index)

tokenizer2.fit_on_texts(train_all['Title'])
x_train_Title_tokens = tokenizer2.texts_to_sequences(x_train['Title'])
x_test_Title_tokens = tokenizer2.texts_to_sequences(x_test['Title'])
x_train_Title_pad = pad_sequences(x_train_Title_tokens,maxlen=max_length2,padding='post')
x_test_Title_pad = pad_sequences(x_test_Title_tokens,maxlen=max_length2,padding='post')

unique_words = len(tokenizer.word_index)+1

tokenizer3.fit_on_texts(train_all['Author'])
x_train_Author_tokens = tokenizer3.texts_to_sequences(x_train['Author'])
x_test_Author_tokens = tokenizer3.texts_to_sequences(x_test['Author'])
x_train_Author_pad = pad_sequences(x_train_Author_tokens,maxlen=max_length3,padding='post')
x_test_Author_pad = pad_sequences(x_test_Author_tokens,maxlen=max_length3,padding='post')

unique_words = len(tokenizer.word_index)+1

print(x_train_Article_pad.shape)
print(x_train_Title_pad.shape)
print(x_train_Author_pad.shape)
print(x_test_Article_pad.shape)
print(x_test_Title_pad.shape)
print(x_test_Author_pad.shape)
print('Found %d unique words.' % len(tokenizer.word_index))
print('Found %d unique words.' % len(tokenizer2.word_index))
print('Found %d unique words.' % len(tokenizer3.word_index))

(13092, 3107)
(13092, 34)
(3273, 3107)
(3273, 34)


In [ ]:
import os
import numpy as np
embeddings_index = {}
#GLOVE_DIR = 'glove.6B/'
f = open('/content/drive/My Drive/glove.6B.50d.txt',encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
word_index = tokenizer.word_index
word_index2 = tokenizer2.word_index
word_index3 = tokenizer3.word_index
embedding_matrix = np.zeros((len(word_index) + 1, 50))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
embedding_matrix2 = np.zeros((len(word_index2) + 1, 50))
for word, i in word_index2.items():
    embedding_vector2 = embeddings_index.get(word)
    if embedding_vector2 is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix2[i] = embedding_vector2
embedding_matrix3 = np.zeros((len(word_index3) + 1, 50))
for word, i in word_index3.items():
    embedding_vector3 = embeddings_index.get(word)
    if embedding_vector3 is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix3[i] = embedding_vector3

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout, Embedding, BatchNormalization, Activation,Concatenate,Input,LSTM,Bidirectional

#y_train = y_train.to_numpy()
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)
#with tf.device('/cpu:0'):
#model = keras.models.Sequential()
inputAr = Input(shape=max_length)
input_Article = Embedding(len(word_index) + 1,50,weights=[embedding_matrix],
                          trainable=False,name='Article')(inputAr)
#hidden1 = LSTM(100,return_sequences=True)(input_Article)
hidden2 = Bidirectional(LSTM(231,dropout=0.1,recurrent_dropout=0.1))(input_Article)
dense3 = Dense(200,kernel_initializer='he_normal')(hidden2)

inputT = Input(shape=max_length2)
input_Title = Embedding(len(word_index2) + 1,50,weights=[embedding_matrix2],
                        trainable=False,name='Title')(inputT)

#hidden1_2 = LSTM(20,return_sequences=True)(input_Title)
hidden2_2 = Bidirectional(LSTM(46,dropout=0.1,recurrent_dropout=0.1))(input_Title)
dense3_2 = Dense(50,kernel_initializer='he_normal')(hidden2_2)

inputAu = Input(shape=max_length3)
input_Author = Embedding(len(word_index3) + 1,50,weights=[embedding_matrix3],
                          trainable=False,name='Author')(inputAu)
#hidden1_3 = LSTM(5,return_sequences=True)(input_Author)
hidden2_3 = Bidirectional(LSTM(10,dropout=0.1,recurrent_dropout=0.1))(input_Author)
dense3_3 = Dense(20,kernel_initializer='he_normal')(hidden2_3)

concat = keras.layers.concatenate([dense3,dense3_2,dense3_3])

#batch_norm = BatchNormalization()(concat)
dropout = Dropout(0.6)(concat)
dense = Dense(200,kernel_initializer='he_normal')(dropout)
batch_norm2 = BatchNormalization()(dense)
activation = Activation('relu')(batch_norm2)
dense2 = Dense(50,kernel_initializer='he_normal')(activation)
batch_norm3 = BatchNormalization()(dense2)
activation2 = Activation('relu')(batch_norm3)
dropout2 = Dropout(0.6)(activation2)
output = Dense(2, activation='softmax')(dropout2)
model = keras.Model(inputs=[inputAr, inputT, inputAu], outputs=[output])

model.summary()
model.compile(optimizer='RMSprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit((x_train_Article_pad, x_train_Title_pad, x_train_Author_pad), y_train, epochs=20,validation_split=0.2,batch_size=64)

In [ ]:
model.evaluate((x_test_Article_pad, x_test_Title_pad, x_test_Author_pad),y_test.to_numpy())

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
#plt.gca().set_ylim(0, 1)
plt.show()